In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import mf_preprocessor
import importlib
importlib.reload(mf_preprocessor)
from mf_preprocessor import *

In [2]:
#.config("spark.executor.memoryOverhead", "64g")\
spark = SparkSession\
    .builder\
    .appName("mf-preprocessor")\
    .config("spark.executor.heartbeatInterval", "60s")\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.driver.maxResultSize", "64g")\
    .config("spark.sql.crossJoin.enabled", True)\
    .getOrCreate()
sc = spark.sparkContext

In [3]:
def to_pd(df, take=0):
    if take > 0:
        return pd.DataFrame(df.take(take), columns=df.columns)
    return pd.DataFrame(df.collect(), columns=df.columns)

In [4]:
preproc = mf_preprocessor(spark, sc)
df_train = preproc.read_preprocessed("train_mf_indexed.tsv")

In [20]:
df_test = preproc.read_preprocessed("test_mf_indexed.tsv")

In [5]:
to_pd(df_train, 5)

,like,reply,retweet,retweet_with_comment,tweet_id_index,engaging_user_id_index
0,0,0,0,0,55204155,0
1,1,0,0,0,45145377,0
2,0,0,0,0,7081791,0
3,0,0,0,0,37895242,0
4,1,0,0,0,36986637,179


In [8]:
index_files = {"tweet_id": "tweet_id_indices.tsv", "engaging_user_id": "engaging_user_id_indices.tsv"}
id_columns = ["tweet_id", "engaging_user_id"]
id_indices = {}
for id_column in id_columns:
    id_indices[id_column] = spark.read.csv(path=index_files[id_column], sep="\x01", header=True)
    id_indices[id_column] = id_indices[id_column].withColumn(id_column + "_index", F.col(id_column + "_index").cast(LongType()))

In [9]:
to_pd(id_indices["tweet_id"], 5)

,tweet_id,tweet_id_index
0,3320617E253DFAB4C010DB10C68168FF,13260133
1,332061AD7951EBA08A7E410391ECAA79,13260134
2,332061C6A87568C43CDBB5D9CF72210B,13260135
3,33206251F82FE379DA753DA6528D54A6,13260136
4,3320625E223B2766B397657B751EBC8D,13260137


In [21]:
missing_tweet_test = df_test.select("tweet_id_index").distinct()\
    .subtract(df_train.select("tweet_id_index").distinct())
missing_user_test = df_test.select("engaging_user_id_index").distinct()\
    .subtract(df_train.select("engaging_user_id_index").distinct())

In [22]:
missing_tweet_test.count(), missing_user_test.count()
#(8671930, 1787256)

(8671930, 1787256)

In [12]:
missing_tweet_indices = id_indices["tweet_id"].select("tweet_id_index").distinct()\
    .subtract(df_train.select("tweet_id_index").distinct())
missing_user_indices = id_indices["engaging_user_id"].select("engaging_user_id_index").distinct()\
    .subtract(df_train.select("engaging_user_id_index").distinct())

In [13]:
to_pd(missing_tweet_indices, 5)

,tweet_id_index
0,1950
1,2040
2,2214
3,2250
4,4590


In [14]:
to_pd(missing_user_indices, 5)

,engaging_user_id_index
0,26
1,4894
2,5409
3,9945
4,11567


In [15]:
missing_tweet_indices.count(), missing_user_indices.count()

(8671930, 1787256)

In [19]:
tweets_join_index = missing_tweet_indices.rdd.zipWithIndex().toDF()
tweets_join_index = tweets_join_index.withColumn("tweet_id_index", F.col("_1")["tweet_id_index"])\
                .select(F.col("tweet_id_index"), F.col("_2").alias("join_index"))
tweets_join_index.show()

+--------------+----------+
|tweet_id_index|join_index|
+--------------+----------+
|          1950|         0|
|          2040|         1|
|          2214|         2|
|          2250|         3|
|          4590|         4|
|          5385|         5|
|          5556|         6|
|         11625|         7|
|         13460|         8|
|         13723|         9|
|         14719|        10|
|         15194|        11|
|         18147|        12|
|         21965|        13|
|         22165|        14|
|         23116|        15|
|         23506|        16|
|         25084|        17|
|         25826|        18|
|         26486|        19|
+--------------+----------+
only showing top 20 rows



In [23]:
user_join_index = missing_user_indices.rdd.zipWithIndex().toDF()
user_join_index = user_join_index.withColumn("engaging_user_id_index", F.col("_1")["engaging_user_id_index"])\
                .select(F.col("engaging_user_id_index"), F.col("_2").alias("join_index"))
user_join_index.show()

+----------------------+----------+
|engaging_user_id_index|join_index|
+----------------------+----------+
|                    26|         0|
|                  4894|         1|
|                  5409|         2|
|                  9945|         3|
|                 11567|         4|
|                 11625|         5|
|                 14719|         6|
|                 14846|         7|
|                 15057|         8|
|                 15194|         9|
|                 17043|        10|
|                 21965|        11|
|                 29841|        12|
|                 40557|        13|
|                 40634|        14|
|                 50049|        15|
|                 50221|        16|
|                 57157|        17|
|                 60691|        18|
|                 64317|        19|
+----------------------+----------+
only showing top 20 rows



In [24]:
joined = tweets_join_index.join(user_join_index, ["join_index"]).drop("join_index")
joined.show()

+--------------+----------------------+
|tweet_id_index|engaging_user_id_index|
+--------------+----------------------+
|         37261|                 86453|
|         41424|                 89041|
|        744935|               1345001|
|       1420376|               2731065|
|       2524015|               4568705|
|       2558714|               4614151|
|       2706465|               4935442|
|       2929931|               5323077|
|       3068488|               5586670|
|       3325005|               6069962|
|       3375070|               6168921|
|       3731384|               6747533|
|       3828665|               6917052|
|       3856519|               6992119|
|       4477641|               8047243|
|       4719431|               8414496|
|       5358906|               9541598|
|       5745189|              10283025|
|       6995846|              12504305|
|       7362356|              13135878|
+--------------+----------------------+
only showing top 20 rows



In [34]:
joined_schema = joined.withColumn("retweet", F.lit(0).cast(ByteType()))\
    .withColumn("like", F.lit(0).cast(ByteType()))\
    .withColumn("reply", F.lit(0).cast(ByteType()))\
    .withColumn("retweet_with_comment", F.lit(0).cast(ByteType()))
joined_schema.show()

+--------------+----------------------+-------+----+-----+--------------------+
|tweet_id_index|engaging_user_id_index|retweet|like|reply|retweet_with_comment|
+--------------+----------------------+-------+----+-----+--------------------+
|         37261|                 86453|      0|   0|    0|                   0|
|         41424|                 89041|      0|   0|    0|                   0|
|        744935|               1345001|      0|   0|    0|                   0|
|       1420376|               2731065|      0|   0|    0|                   0|
|       2524015|               4568705|      0|   0|    0|                   0|
|       2558714|               4614151|      0|   0|    0|                   0|
|       2706465|               4935442|      0|   0|    0|                   0|
|       2929931|               5323077|      0|   0|    0|                   0|
|       3068488|               5586670|      0|   0|    0|                   0|
|       3325005|               6069962| 

In [35]:
union_df = df_train.union(joined_schema)
union_df.show()

+----+-----+-------+--------------------+--------------+----------------------+
|like|reply|retweet|retweet_with_comment|tweet_id_index|engaging_user_id_index|
+----+-----+-------+--------------------+--------------+----------------------+
|   0|    0|      0|                   0|      55204155|                     0|
|   1|    0|      0|                   0|      45145377|                     0|
|   0|    0|      0|                   0|       7081791|                     0|
|   0|    0|      0|                   0|      37895242|                     0|
|   1|    0|      0|                   0|      36986637|                   179|
|   1|    0|      0|                   0|      23840070|                   179|
|   0|    0|      0|                   0|      45630796|                   179|
|   1|    0|      0|                   0|      42699687|                   179|
|   1|    0|      0|                   0|      19320798|                   179|
|   1|    0|      1|                   0

In [36]:
#1787256
union_df.count() - df_train.count()

1787256